## ChatGPT

In [6]:
import pandas as pd

queries = pd.read_csv("../Data/Queries_IT_final.csv")
queries.head()

,Query,Source
0,Chi era il padre di Micerino ?,IT-GUI
1,Il figlio di Chefren,IT-GUI
2,Il padre di Micerino,IT-GUI
3,Micerino,IT-GUI
4,Quanto e alta la tomba di Cheope?,IT-GUI


In [7]:
type(queries)

pandas.core.frame.DataFrame

In [8]:
from openai import OpenAI
import json
from tqdm.notebook import tqdm
from datetime import date

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["chatgpt"]["api_key"]
f.close()

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

# query = "Hello"

model_id = 'gpt-3.5-turbo'

gpt_resp_list = []

for query in tqdm(queries["Query"], total=len(queries)):

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Follow these two instructions in all your responses: 1. Use Italian language only; 2. Do not use English except in programming language if any."
            },
            {
                "role": "user",
                "content": query,
            }
        ],
            model=model_id,
        )

    gpt_resp = response.choices[0].message.content
    gpt_resp_list.append(gpt_resp)

queries["GPT"] = gpt_resp_list
queries["date_generated"] = [date.today()]*len(queries)

queries.head()


  0%|          | 0/110 [00:00<?, ?it/s]

,Query,Source,GPT,date_generated
0,Chi era il padre di Micerino ?,IT-GUI,Il padre di Micerino era Chefren.,2024-10-07
1,Il figlio di Chefren,IT-GUI,"Il figlio di Chefren era Cheope, il faraone ch...",2024-10-07
2,Il padre di Micerino,IT-GUI,"Il padre di Micerino era Chefren, faraone dell...",2024-10-07
3,Micerino,IT-GUI,Micerino è stato un faraone dell'Antico Egitto...,2024-10-07
4,Quanto e alta la tomba di Cheope?,IT-GUI,"La tomba di Cheope, la più grande delle tre pi...",2024-10-07


In [9]:
queries.to_csv("../Data/GPT_response.csv", index=False)

## Google Gemma

In [13]:
from huggingface_hub import login
import json
import pandas as pd

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["hugging_face"]["api_key"]
f.close()


login(token=API_KEY)

queries = pd.read_csv("../Data/Queries_IT_final.csv")
queries.head()

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Hrishita Chakrabarti\.cache\huggingface\token
Login successful


,Query,Source
0,Chi era il padre di Micerino ?,IT-GUI
1,Il figlio di Chefren,IT-GUI
2,Il padre di Micerino,IT-GUI
3,Micerino,IT-GUI
4,Quanto e alta la tomba di Cheope?,IT-GUI


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

def create_pipe(model_name):

  # Specify the LLM model we'll be using
  
  # Configure for GPU usage
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      device_map="auto",
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
  )
  
  # Load the tokenizer for the chosen model
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  # Create a pipeline object for easy text generation with the LLM
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

  return pipe

def gen_resp(pipe, query):
  """Sends a conversation history to the AI assistant and returns the answer.
  
  Args:
    messages (list): A list of dictionaries, each with "role" and "content" keys.
  
  Returns:
    str: The answer from the AI assistant.
  """ 

  messages = [
    # {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": query}
  ]

  generation_args = {
      "max_new_tokens": 256,     # Maximum length of the response
      "return_full_text": False,      # Only return the generated text
  }

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

In [15]:
from tqdm.notebook import tqdm
from datetime import date
import pandas as pd

def create_resp_file(model, model_name):
  print("Model name: ", model_name)

  pipe = create_pipe(model)
  queries = pd.read_csv("/content/drive/MyDrive/SOL/Queries_IT_final.csv")

  LLM_resp = []

  for query in tqdm(queries["Query"], total=len(queries)):

  # query = "What about solving the equation 2x + 3 = 7?"

    result = gen_resp(pipe, query)
    LLM_resp.append(result)

  col_name = model_name + "_resp"

  queries[col_name] = LLM_resp
  queries["date_generated"] = [date.today()]*len(queries)
  file_name = "../Data/" + model_name + "_response.csv"
  queries.to_csv(file_name, index=False)

In [ ]:
model_options = ["google/gemma-2b-it"]
model_names = ["Gemma_2b"]

model = model_options[0]
model_name = model_names[0]
create_resp_file(model, model_name)